In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
import os

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras import Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

## Exploratory Visualization and Image Analysis

In [ ]:
data_path = 'C:\\Users\\alvar\\Documents\\tf_templates\\DATA\\cell_images'
train_path = data_path + '\\train\\'
test_path = data_path + '\\test\\'

In [ ]:
os.listdir(test_path)

In [ ]:
os.listdir(train_path)

In [ ]:
uninfected_cell = train_path + '\\uninfected\\' + os.listdir(train_path + '\\uninfected')[0]
uninfected_img = imread(uninfected_cell)
plt.imshow(uninfected_img);

In [ ]:
parasitized_cell = train_path + '\\parasitized\\' + os.listdir(train_path + '\\parasitized')[0]
parasitized_img = imread(parasitized_cell)
plt.imshow(parasitized_img);

In [ ]:
parasitized_img.shape

In [ ]:
uninfected_img.shape

In [ ]:
dim1 = []
dim2 = []
for image_filename in os.listdir(test_path+'\\uninfected'):
    
    img = imread(test_path+'\\uninfected'+'\\'+image_filename)
    d1,d2,colors = img.shape
    dim1.append(d1)
    dim2.append(d2)

In [ ]:
sns.jointplot(x=dim1, y=dim2);

In [ ]:
np.mean(dim1)

In [ ]:
np.mean(dim2)

In [ ]:
img_height = 131
img_width = 131

In [ ]:
parasitized_img.max()

In [ ]:
image_gen = ImageDataGenerator(
    rotation_range=(-20,20),
    width_shift_range=0.1,
    height_shift_range=0.1,
    # rescale=1./255,
    shear_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [ ]:
image_gen.flow_from_directory(train_path)

In [ ]:
image_gen.flow_from_directory(test_path)

## Creating the Model and Training

In [ ]:
model = Sequential()

model.add(Input(shape=(img_height, img_width, 3)))

model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
train = image_gen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    color_mode='rgb',
    batch_size=32,
    class_mode='binary'
)

In [ ]:
test = image_gen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    color_mode='rgb',
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

In [ ]:
train.class_indices

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
model.fit(train, epochs=20, validation_data=test, callbacks=[early_stop])

## Save Model

In [ ]:
model.save('C:\\Users\\alvar\\Documents\\tf_templates\\CNN\\malaria_model.keras')

## Evaluation

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses[['accuracy','val_accuracy']].plot();

In [ ]:
losses[['loss','val_loss']].plot();

In [ ]:
print(model.metrics_names)
print(model.evaluate(test, verbose=1))

In [ ]:
predictions = np.round(model.predict(test))
predictions

In [ ]:
y_true = test.classes
y_true

In [ ]:
print(classification_report(y_true,predictions))

## Predict Classes

In [ ]:
my_image = load_img(parasitized_cell, target_size=(img_height, img_width, 3))
my_image = np.expand_dims(img_to_array(my_image), axis=0)
my_image.shape

In [ ]:
np.round(model.predict(my_image))

In [ ]:
train.class_indices